#   Антагонистическая игра 5x5

<img width="50%" src="gt13.1.jpg" alt="problem 1 statement" />

Обозначим матрицей $P$ таблицу `13.1`.
Задача первого игрока состоит в нахождении:
\begin{equation}\label{obj}
\max_x \min_y x^\mathrm{T} P y,
\end{equation}
а задача второго в
\begin{equation}\label{player2obj}
\max_y \min_x y^\mathrm{T} Q x,
\end{equation}
где $$Q = J - P^\mathrm{T}.$$

Цель второго игрока можно переформулировать в традиционном
для zero-sum игр виде:
\begin{equation}\label{player2obj2}
\min_y \max_x x^\mathrm{T} P y.
\end{equation}
Очевидно задачи \eqref{player2obj} и \eqref{player2obj2}
имеют одинаковое решение.
Однако задача \eqref{player2obj} лучше соответствует доменной области
(это оптимизация минимальной **доли рынка**, которую захватит второй игрок),
а вторая задача соответствует антагонистической интерпретации.

Выражение \eqref{obj} можно упростить,
с учётом того, что $y_i\in[0,1]$ и $\sum_i y_k = 1$.

При любом фиксированном $X$ имеют место равенства
$$\min_Y X^\mathrm{T}PY = \min_Y\left[\sum_k y_k X^{\mathrm{T}}P e_k\right] =
\min_i X^{\mathrm{T}}P e_i.$$

Другими словами, если $V_\star$ --- оптимальное решение, то
$$X^{\mathrm{T}}P_{[:,i]} \geq V_\star\ \text{для всех}\ i{=}\overline{1,n},$$
где $P_{[:,i]} = P e_i$ --- $i$-й столбец матрицы $P$.

Теперь задачу можно переписать в виде:
$$V\longrightarrow\max,$$
$$P^{\mathrm{T}} X \geq V,\ \mathbf{1}^{\mathrm{T}}X = 1,\ X\geq 0.$$
Неравенство $P^{\mathrm{T}} X \geq V$ понимается поэлементно,
а $\mathbf{1}$ означает вектор, составленный из единиц.

Переместим $V$ в левую часть уравнения,
введём slack переменные $S = \begin{pmatrix}s_1\\ \vdots \\ s_n\end{pmatrix}\geq 0$
и получим следующую линейную программу:
$$-V\to\inf,$$
$$-V\mathbf{1} - S - P^{\mathrm{T}}X = 0,$$
$$\mathbf{1}^\mathrm{T}X = 1.$$

Эквивалентно:
$$
\left\{
\begin{aligned}
& {-}V\to\inf,\\
&
\left(
\begin{array}{c|c|c}
\begin{matrix}
-1 \\ \vdots \\ -1
\end{matrix}
&
\begin{matrix}
-1 & & 0 \\
   & \ddots & \\
0  & & -1
\end{matrix} &
P^{\mathrm{T}} \\ \hline
\begin{matrix}
0
\end{matrix}
&
\begin{matrix}
0 & \cdots & 0
\end{matrix}
&
\begin{matrix}
1 & \cdots & 1
\end{matrix}
\end{array}
\right)
\begin{pmatrix}
V\\
s_1\\
\vdots\\
s_n\\
x_1\\
\vdots\\
x_n
\end{pmatrix}
=\begin{pmatrix}
0\\
\vdots\\
0\\ \hline
1
\end{pmatrix}
,\\
& V,\ S,\ X \geq 0.
\end{aligned}
\right.$$

In [1]:
P = [
    .5 .5 .4 .5 .21;
    .5 .4 .7 .11 .6;
    .21 .3 .4 .1 .7;
    .3 .6 .7 .3 .2;
    .4 .4 .3 .01 .2
];
DEMAND = 1000 + 1;

function [X, V] = solve_antagonistic (P)
    [m, n] = size(P);
    A = [-ones(n,1), -eye(n), P'; zeros(1,n+1), ones(1,m)];
    b = [zeros(n,1);1];
    c = [1; zeros(size(A,2)-1,1)];
    X = glpk(-c,A,b);
    V = c'*X;
    X = X(n+2:end);
endfunction


[x, V1] = solve_antagonistic(P);
[y, V2] = solve_antagonistic(1-P');

Убедимся, что найдена седловая точка ($V_2 = 1 - V_1$):

In [2]:
V1 + V2

ans =  1


Итак, решение равновесное.
Определим, сколько товаров производить первому игроку:

In [3]:
x' * V1 * DEMAND

ans =

   249.46017     0.00000   120.57242     0.00000     0.00000



Второму:

In [4]:
y' * V2 * DEMAND

ans =

     0.00000     0.00000     0.00000   347.38655   283.58086



In [5]:
first = round(x' * V1 * DEMAND)
second =  round(y' * V2 * DEMAND)
sum(first+second)

first =

   249     0   121     0     0

second =

     0     0     0   347   284

ans =  1001
